In [1]:
# 사용할 패키지 불러오기
import pandas as pd
import numpy as np
np.set_printoptions(threshold=np.inf) # print all numpy values

# For deep learning model 
import keras
from keras.layers import Dense, Input, concatenate, Dropout
from keras.models import Sequential, Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras import metrics
from keras import backend as K

Using TensorFlow backend.


In [2]:
# 데이터 준비하기 - 학습 데이터, 테스트 데이터 로드
categorical = ['발생지시도', '발생지시군구', '사고유형_대분류', '사고유형_중분류', '법규위반', 
            '도로형태_대분류', '도로형태', '당사자종별_1당_대분류', '당사자종별_2당_대분류', '주야', '요일']
numerical = ['사상자수', '사망자수', '중상자수', '경상자수','부상신고자수']

x_train_num = pd.read_csv('./교통사망사고정보/Kor_Train_교통사망사고정보(12.1~17.6).csv',encoding='cp949', usecols=numerical)

x_train_cat = pd.read_csv('./교통사망사고정보/Kor_Train_교통사망사고정보(12.1~17.6).csv',encoding='cp949', usecols=categorical)

x_test = pd.read_csv('./test_kor.csv', encoding='cp949')

x_test_num = pd.read_csv('./test_kor.csv', encoding='cp949', usecols=numerical)

x_test_cat = pd.read_csv('./test_kor.csv', encoding='cp949', usecols=categorical)

In [3]:
# one-hot encoding

# 모든 존재하는 column의 one hot encoding을 위해 train과 test의 카테고리 통합
all_data = pd.concat((x_test_cat.dropna(), x_train_cat))

# get_dummies가 가능하도록 type 설정
for column in all_data.select_dtypes(include=[np.object]).columns:
    x_train_cat[column] = x_train_cat[column].astype('category', categories = all_data[column].unique())
    x_test_cat[column] = x_test_cat[column].astype('category', categories = all_data[column].unique())

# 모든 column에 대해 one hot encoding 수행
x_train_cat = pd.get_dummies(data=x_train_cat)
x_test_cat = pd.get_dummies(data=x_test_cat)

/home/iron/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  
/home/iron/.local/lib/python3.5/site-packages/ipykernel_launcher.py:9: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  if __name__ == '__main__':


In [4]:
# Train Data Shape check
print(x_train_cat.shape)
print(x_train_num.shape)

# Test Data Shape check
print(x_test_cat.shape)
print(x_test_num.shape)

(25037, 328)
(25037, 5)
(50, 328)
(50, 5)


In [5]:
# test file에 있는 값을 result file에 저장

def save_result(sheet_name_csv):
    result_file_address = './result_kor.csv'
    test_file = pd.read_csv(sheet_name_csv, encoding='cp949', names= [chr(y) for y in range(ord('A'),ord('P')+1)])
    result_file = pd.read_csv(result_file_address, encoding='cp949')
    print(sheet_name_csv)
    
    cols = result_file['열'].astype('str').values
    rows = result_file['행'].astype('str').astype('int').values
    vals = result_file['값'].astype('str').values

    for i, (row, col) in enumerate(zip(rows, cols)):      
        vals[i] = test_file[col][int(row)-1]


    with open(result_file_address, 'wb+') as f:
        np.savetxt(f,  np.c_[rows,cols,vals],  delimiter=",", fmt='%s', encoding='cp949', header='행,열,값', comments='')
    print('Save Success')

In [6]:
# Hyper Parameters 
epochs = 20
batch_size = 128
optimizer = 'adam' 

In [7]:
# Numeric case를 학습하고 예상값을 return

# case: column명 array
# start: 예측할 시작 row
# end: 예측할 마지막 row
def numeric_case(case, start, end):
    
    K.clear_session()
    case_copy=case.copy()
    
    # Case 확인
    print("Case:", case)
    
    # Train Data
    X = x_train_num.drop(columns=case)
    X = pd.concat([X, x_train_cat], axis=1).values
    
    # Test Data
    X_test = x_test_num.drop(columns=case)
    X_test = pd.concat([X_test, x_test_cat],axis=1).values
    
    # Label Data
    if '사상자수' in case:       
        case_copy.remove('사상자수')
        print('사상자제거:', case_copy) 
    Y = x_train_num[case_copy].values
    
    
    # Model define(Ver. MP)
    num_input = Input(shape=(len(X[0]),), name='num_input')
    x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(num_input)
    x = Dropout(0.3)(x)
    x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
    x = Dropout(0.3)(x)
    num_output = Dense(len(Y[0]), name='num_output')(x)

    model = Model(inputs=num_input, outputs=num_output)

    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics=['accuracy'])

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=10, verbose=1, factor=0.5, min_lr=0.00000001)

    callbacks = [
        learning_rate_reduction, # learning_rate를 점차 감소시킴으로서 최적값에 접근하는 방식
        EarlyStopping('val_loss', patience=15)# val_loss이 최적값에서 멀어지는 순간 epoch가 남아도 학습 중지
    ]
    
    history = model.fit(X, Y, epochs=epochs, batch_size=batch_size, callbacks=callbacks, validation_split=0.2 )
    
    
    # make a prediction
    Y_test = model.predict(X_test[start:end+1])
    
    del model
    
    return Y_test

In [8]:
# Categorical case를 학습하고 예상값을 return

# case: column명 array
# start: 예측할 시작 row
# end: 예측할 마지막 row
def categorical_case(case, start, end):
    
    # 기존 session 초기화
    K.clear_session()
    
    # Case 확인
    print("Case:", case)
    
    col_name = [] # ex. '사고유형_대분류_차대차', '사고유형_대분류_차대사람', '사고유형_대분류_차량단독'
    label_name = [] #  ex. '차대차', '차대사람', '차량단독
    
    # One Hot Encoding 후 Columns 이름과 Columns에 들어 있는 값 
    for col in case:
        label_name.extend(all_data[col].unique()) 
        for name in all_data[col].unique():
            col_name.append(col+'_'+name)

    # Train Data 
    X = x_train_cat.drop(columns=col_name)
    X = pd.concat([X, x_train_num], axis=1).values

    # Test Data
    X_test = x_test_cat.drop(columns=col_name)
    X_test = pd.concat([X_test, x_test_num],axis=1).values
    
    # Label Data
    Y = x_train_cat[col_name].values
    
    # Model define(Ver. MP)
    cat_input = Input(shape=(len(X[0]),), name='cat_input')
    x = Dense(512, activation='relu')(cat_input)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    cat_output = Dense(len(Y[0]), activation='sigmoid', name='cat_output')(x)

    model = Model(inputs=cat_input, outputs=cat_output)

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=10, verbose=1, factor=0.5, min_lr=0.00000001)

    callbacks = [
        learning_rate_reduction, # learning_rate를 점차 감소시킴으로서 최적값에 접근하는 방식
        EarlyStopping('val_loss', patience=15), # val_loss이 최적값에서 멀어지는 순간 epoch가 남아도 학습 중지
        ]

    history = model.fit(X, Y, epochs=epochs, batch_size=batch_size, callbacks=callbacks,validation_split=0.2 )
    
    # make a prediction
    Y_test = model.predict(X_test[start:end+1])
    

    '''
    예시 출력:
        사고유형_대분류 : 차량단독
        사고유형_중분류 : 공작물충돌
        법규위반 : 안전운전 의무 불이행
    '''
    result = []
    for cat in Y_test: 
        x_list = list(cat)
        label_name_x = label_name.copy()
        temp = []
        for col in case:
#             print(col, ':', label_name_x[x_list.index(max(x_list[0:len(all_data[col].unique())]))] )
            temp.append(label_name_x[x_list.index(max(x_list[0:len(all_data[col].unique())]))])
            del x_list[:len(all_data[col].unique())]
            del label_name_x[:len(all_data[col].unique())]
        result.append(temp)
        
        
    del model
    
    return np.array(result)


In [9]:
# Mix case를 학습하고 예상값을 return

# case: column명 array
# n: 범주형 데이터 수
# start: 예측할 시작 row
# end: 예측할 마지막 row
def mix_case(case, n, start, end):

    # 기존 session 초기화
    K.clear_session()
    
    # Case 확인
    print("Case:", case)
    
    # list 값 복사
    case_copy = case.copy()
    
    # categorical cases 처리
    col_name = []
    label_name = []
    cat_name = []
#     cat_name = case_copy[-n:]
    
    for col in case_copy:
        if col in categorical:
            cat_name.append(col)
            label_name.extend(all_data[col].unique()) 
            for name in all_data[col].unique():
                col_name.append(col+'_'+name)
                
                
    

    Y_cat = x_train_cat[col_name].values
    X1 = x_train_cat.drop(columns=col_name)
    X_test1 = x_test_cat.drop(columns=col_name)
    

    for remove_name in cat_name:
        case_copy.remove(remove_name)
    
    print('해당 numberical 컬럼 순서', case_copy)
    
    
    # numerical cases 처리
    X2 = x_train_num.drop(columns=case_copy)
    X_test2 = x_test_num.drop(columns=case_copy)
    
    if '사상자수' in case: 
        case_copy.remove('사상자수')
        print('사상자제거:', case_copy) 
    Y_num = x_train_num[case_copy].values
    
    X = pd.concat([X1, X2], axis=1).values
    X_test = pd.concat([X_test1, X_test2],axis=1).values
    
    
    # Model define(Ver.MP)
    cat_input = Input(shape=(len(X[0]),), name='cat_input')
    x = Dense(1024, activation='relu')(cat_input)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    if n == 1:
        cat_output = Dense(len(Y_cat[0]), activation='softmax', name='cat_output')(x)
    elif n == 2:
        cat_output = Dense(len(Y_cat[0]), activation='sigmoid', name='cat_output')(x)
        
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    num_output = Dense(len(Y_num[0]), name='num_output')(x)

    model = Model(inputs=cat_input, outputs=[cat_output, num_output])

    if n==1:
        model.compile(optimizer=optimizer,
                      loss={'cat_output': 'categorical_crossentropy', 'num_output': 'mse'},
                      metrics=['accuracy'])
    elif n==2:
        model.compile(optimizer=optimizer,
                      loss={'cat_output': 'binary_crossentropy', 'num_output': 'mse'},
                      metrics=['accuracy'])
    
    
    learning_rate_reduction = ReduceLROnPlateau(monitor='cat_output_acc', 
                                            patience=10, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00000001)

    callbacks = [
        learning_rate_reduction, # learning_rate를 점차 감소시킴으로서 최적값에 접근하는 방식
        EarlyStopping('val_loss', patience=20), # val_loss이 최적값에서 멀어지는 순간 epoch가 남아도 학습 중지
    ]

    history = model.fit(X, {'cat_output':Y_cat, 'num_output':Y_num}, epochs=epochs, batch_size=batch_size, callbacks=callbacks,validation_split=0.2 )
    
    
    # make a prediction
    Y_test = model.predict(X_test[start:end+1])    
    
    result = []
    for cat, num in zip(Y_test[0], Y_test[1]):
        x_list = list(cat)
        label_name_x = label_name.copy()
        temp = []
        for col in cat_name:
            temp.append(label_name_x[x_list.index(max(x_list[0:len(all_data[col].unique())]))])
            
            # 출력한 Column과 데이터 삭제
            del x_list[:len(all_data[col].unique())]
            del label_name_x[:len(all_data[col].unique())]
        temp.extend(num)
        result.append(temp)
        
    
    return np.array(result)
    

In [10]:
# set result array to each cases
def setResult(arr, predict, extent, case):
    result_arr = arr
    print('setResult 호출')
        
    for xy in extent:
        if 3 == xy[1]:
            extent.remove(xy)
    
    for i, cell in enumerate(extent):
        result_arr[cell[1]] = predict[0, i]
    
    # 사망자 수는 사망자, 중상자, 경상자, 부상신고자 수에 의해 영향을 받음.
    if '사상자수' in case:
        result_arr[2:7] = np.asarray(result_arr[2:7], dtype='float64')
        result_arr[3] = result_arr[2] + result_arr[4:7].sum()
    
    
    print('result_arr 결과:',result_arr)
    
    return result_arr
    

In [ ]:
np_x_test = x_test.isnull().values
x_test_col_name = x_test.columns.values
new_sheet = np.array([x_test.columns.values])
Case_ex = []
label_ex = []
for row in range(len(np_x_test)):
    temp_case = []
    temp_label = []
    for col in range(len(np_x_test[0])):
        if np_x_test[row, col] == True:
            temp_case.append(x_test_col_name[col])
            temp_label.append([row, col])
    Case_ex.append(temp_case)
    label_ex.append(temp_label)

# print(Case_ex)
# print(label_ex)

for idx, (case, cell) in enumerate(zip(Case_ex, label_ex)):
    print(idx)
    print(cell)
    num = 0
    cat = 0
    for one_case in case:
        if one_case in numerical:
            num += 1
        elif one_case in categorical:
            cat += 1
        
    if len(case) == num:
        Case_prediction = numeric_case(case, cell[0][0], cell[-1][0])
        print("예측값 출력: ", Case_prediction)
        answer = setResult(x_test.loc[cell[0][0]].values, Case_prediction, cell, case)
        

    elif len(case) == cat:
        
        Case_prediction = categorical_case(case, cell[0][0], cell[-1][0])
        print("예측값 출력: ", Case_prediction)
        answer = setResult(x_test.loc[cell[0][0]].values, Case_prediction, cell, case)
        
    else:
        print(cell)
        
        temp_num = []
        temp_cat = []
        for one_case in case:
            if one_case in numerical:
                temp_num.append(cell[case.index(one_case)])
                
            if one_case in categorical:
                temp_cat.append(cell[case.index(one_case)])
                
        temp_cat.extend(temp_num)
        print(temp_cat)
        
        Case_prediction = mix_case(case, cat, cell[0][0], cell[-1][0])
        print("예측값 출력: ", Case_prediction)
        answer = setResult(x_test.loc[cell[0][0]].values, Case_prediction, temp_cat, case)
            
    print()
    new_sheet = np.append(new_sheet, [answer], axis=0)

0
[[0, 2], [0, 3], [0, 5]]
Case: ['사망자수', '사상자수', '경상자수']
사상자제거: ['사망자수', '경상자수']
Train on 20029 samples, validate on 5008 samples
Epoch 1/20
20029/20029 [==============================] - 1s 50us/step - loss: 0.6210 - acc: 0.9365 - val_loss: 0.5286 - val_acc: 0.9345
Epoch 2/20
20029/20029 [==============================] - 0s 24us/step - loss: 0.5769 - acc: 0.9429 - val_loss: 0.5294 - val_acc: 0.9455
Epoch 3/20
20029/20029 [==============================] - 0s 23us/step - loss: 0.5445 - acc: 0.9476 - val_loss: 0.5393 - val_acc: 0.9451
Epoch 4/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.4937 - acc: 0.9462 - val_loss: 0.5458 - val_acc: 0.9373
Epoch 5/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.4820 - acc: 0.9494 - val_loss: 0.5633 - val_acc: 0.9511
Epoch 6/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.4299 - acc: 0.9516 - val_loss: 0.5840 - val_acc: 0.9473
Epoch 7/20
20029/20029 [=======================

Train on 20029 samples, validate on 5008 samples
Epoch 1/20
20029/20029 [==============================] - 1s 27us/step - loss: 0.5264 - acc: 0.8577 - val_loss: 0.9069 - val_acc: 0.8151
Epoch 2/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.5109 - acc: 0.9169 - val_loss: 0.8852 - val_acc: 0.9904
Epoch 3/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.4924 - acc: 0.9522 - val_loss: 0.8847 - val_acc: 0.9882
Epoch 4/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.4588 - acc: 0.9569 - val_loss: 0.8858 - val_acc: 0.9904
Epoch 5/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.4277 - acc: 0.9591 - val_loss: 0.9385 - val_acc: 0.9906
Epoch 6/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.4473 - acc: 0.9709 - val_loss: 0.8822 - val_acc: 0.9906
Epoch 7/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.4175 - acc: 0.9551 - val_loss: 0.8832 - val_acc:

20029/20029 [==============================] - 0s 22us/step - loss: 0.3861 - acc: 0.7694 - val_loss: 1.2130 - val_acc: 0.8614
예측값 출력:  [[1.2452822  0.63196117]]
setResult 호출
result_arr 결과: ['주간' '목' 2.0 3.8772433400154114 1.2452821731567383 0.6319611668586731 0.0
 '경북' '영천시' '차대차' '추돌' '안전운전 의무 불이행' '단일로' '기타단일로' '화물차' '화물차']

6
[[6, 3], [6, 4], [6, 5]]
Case: ['사상자수', '중상자수', '경상자수']
사상자제거: ['중상자수', '경상자수']
Train on 20029 samples, validate on 5008 samples
Epoch 1/20
20029/20029 [==============================] - 1s 27us/step - loss: 1.0934 - acc: 0.6125 - val_loss: 1.0008 - val_acc: 0.8237
Epoch 2/20
20029/20029 [==============================] - 0s 21us/step - loss: 1.0057 - acc: 0.6211 - val_loss: 0.9848 - val_acc: 0.5653
Epoch 3/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.9577 - acc: 0.6120 - val_loss: 1.0057 - val_acc: 0.3802
Epoch 4/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.9202 - acc: 0.6282 - val_loss: 1.0073 - val_acc:

Epoch 16/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.1935 - acc: 0.9548 - val_loss: 0.5200 - val_acc: 0.9275
Epoch 17/20
20029/20029 [==============================] - 0s 21us/step - loss: 0.1830 - acc: 0.9542 - val_loss: 0.5181 - val_acc: 0.9251
Epoch 18/20
20029/20029 [==============================] - 0s 21us/step - loss: 0.1677 - acc: 0.9585 - val_loss: 0.5276 - val_acc: 0.9281
Epoch 19/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.1774 - acc: 0.9567 - val_loss: 0.5266 - val_acc: 0.9253
Epoch 20/20
20029/20029 [==============================] - 0s 22us/step - loss: 0.1807 - acc: 0.9589 - val_loss: 0.5386 - val_acc: 0.9205
예측값 출력:  [[ 1.6590407 13.115829 ]]
setResult 호출
result_arr 결과: ['주간' '일' 1.6590406894683838 39.77487015724182 13.115829467773438 21.0 4.0
 '강원' '평창군' '차대차' '측면충돌' '기타(운전자법규위반)' '단일로' '기타단일로' '건설기계' '승합차']

9
[[9, 2], [9, 3], [9, 4]]
Case: ['사망자수', '사상자수', '중상자수']
사상자제거: ['사망자수', '중상자수']
Train on 20029 samples

20029/20029 [==============================] - 1s 34us/step - loss: 0.0733 - acc: 0.9701 - val_loss: 0.0772 - val_acc: 0.9688
Epoch 10/20
20029/20029 [==============================] - 1s 34us/step - loss: 0.0718 - acc: 0.9709 - val_loss: 0.0779 - val_acc: 0.9679
Epoch 11/20
20029/20029 [==============================] - 1s 34us/step - loss: 0.0711 - acc: 0.9712 - val_loss: 0.0787 - val_acc: 0.9676
Epoch 12/20
20029/20029 [==============================] - 1s 33us/step - loss: 0.0696 - acc: 0.9716 - val_loss: 0.0800 - val_acc: 0.9677
Epoch 13/20
20029/20029 [==============================] - 1s 34us/step - loss: 0.0689 - acc: 0.9722 - val_loss: 0.0807 - val_acc: 0.9671
Epoch 14/20
20029/20029 [==============================] - 1s 34us/step - loss: 0.0673 - acc: 0.9727 - val_loss: 0.0805 - val_acc: 0.9671
Epoch 15/20
20029/20029 [==============================] - 1s 34us/step - loss: 0.0658 - acc: 0.9734 - val_loss: 0.0827 - val_acc: 0.9666
Epoch 16/20
20029/20029 [=====================

...

20029/20029 [==============================] - 1s 34us/step - loss: 0.1065 - acc: 0.9599 - val_loss: 0.1136 - val_acc: 0.9566
Epoch 15/20
20029/20029 [==============================] - 1s 34us/step - loss: 0.1054 - acc: 0.9603 - val_loss: 0.1136 - val_acc: 0.9567
Epoch 16/20
20029/20029 [==============================] - 1s 33us/step - loss: 0.1051 - acc: 0.9604 - val_loss: 0.1138 - val_acc: 0.9564
Epoch 17/20
20029/20029 [==============================] - 1s 34us/step - loss: 0.1041 - acc: 0.9608 - val_loss: 0.1146 - val_acc: 0.9562
Epoch 18/20
20029/20029 [==============================] - 1s 33us/step - loss: 0.1037 - acc: 0.9610 - val_loss: 0.1147 - val_acc: 0.9562
Epoch 19/20
20029/20029 [==============================] - 1s 33us/step - loss: 0.1030 - acc: 0.9614 - val_loss: 0.1152 - val_acc: 0.9562
Epoch 20/20
20029/20029 [==============================] - 1s 34us/step - loss: 0.1028 - acc: 0.9614 - val_loss: 0.1156 - val_acc: 0.9563
예측값 출력:  [['금' '측면직각충돌' '신호위반' '교차로']]
setResu

Epoch 3/20
20029/20029 [==============================] - 1s 54us/step - loss: 0.3328 - cat_output_loss: 0.2543 - num_output_loss: 0.0785 - cat_output_acc: 0.8780 - num_output_acc: 0.9643 - val_loss: 0.3326 - val_cat_output_loss: 0.2504 - val_num_output_loss: 0.0822 - val_cat_output_acc: 0.8796 - val_num_output_acc: 0.9629
Epoch 4/20
20029/20029 [==============================] - 1s 54us/step - loss: 0.3227 - cat_output_loss: 0.2488 - num_output_loss: 0.0739 - cat_output_acc: 0.8815 - num_output_acc: 0.9655 - val_loss: 0.3262 - val_cat_output_loss: 0.2473 - val_num_output_loss: 0.0789 - val_cat_output_acc: 0.8814 - val_num_output_acc: 0.9629
Epoch 5/20
20029/20029 [==============================] - 1s 54us/step - loss: 0.3143 - cat_output_loss: 0.2444 - num_output_loss: 0.0699 - cat_output_acc: 0.8841 - num_output_acc: 0.9657 - val_loss: 0.3214 - val_cat_output_loss: 0.2457 - val_num_output_loss: 0.0757 - val_cat_output_acc: 0.8835 - val_num_output_acc: 0.9629
Epoch 6/20
20029/20029 [=

Epoch 7/20
20029/20029 [==============================] - 1s 54us/step - loss: 0.3053 - cat_output_loss: 0.2390 - num_output_loss: 0.0664 - cat_output_acc: 0.8881 - num_output_acc: 0.9660 - val_loss: 0.3156 - val_cat_output_loss: 0.2460 - val_num_output_loss: 0.0697 - val_cat_output_acc: 0.8813 - val_num_output_acc: 0.9627
Epoch 8/20
20029/20029 [==============================] - 1s 54us/step - loss: 0.3016 - cat_output_loss: 0.2358 - num_output_loss: 0.0658 - cat_output_acc: 0.8902 - num_output_acc: 0.9650 - val_loss: 0.3107 - val_cat_output_loss: 0.2469 - val_num_output_loss: 0.0638 - val_cat_output_acc: 0.8817 - val_num_output_acc: 0.9627
Epoch 9/20
20029/20029 [==============================] - 1s 54us/step - loss: 0.2928 - cat_output_loss: 0.2325 - num_output_loss: 0.0603 - cat_output_acc: 0.8928 - num_output_acc: 0.9663 - val_loss: 0.3131 - val_cat_output_loss: 0.2468 - val_num_output_loss: 0.0663 - val_cat_output_acc: 0.8815 - val_num_output_acc: 0.9619
Epoch 10/20
20029/20029 [

Epoch 11/20
20029/20029 [==============================] - 1s 54us/step - loss: 0.7934 - cat_output_loss: 0.1264 - num_output_loss: 0.6670 - cat_output_acc: 0.9370 - num_output_acc: 0.8248 - val_loss: 1.1551 - val_cat_output_loss: 0.1234 - val_num_output_loss: 1.0318 - val_cat_output_acc: 0.9392 - val_num_output_acc: 0.8191
Epoch 12/20
20029/20029 [==============================] - 1s 54us/step - loss: 0.7847 - cat_output_loss: 0.1249 - num_output_loss: 0.6598 - cat_output_acc: 0.9373 - num_output_acc: 0.8270 - val_loss: 1.1142 - val_cat_output_loss: 0.1219 - val_num_output_loss: 0.9923 - val_cat_output_acc: 0.9391 - val_num_output_acc: 0.7969
Epoch 13/20
20029/20029 [==============================] - 1s 54us/step - loss: 0.6451 - cat_output_loss: 0.1246 - num_output_loss: 0.5205 - cat_output_acc: 0.9377 - num_output_acc: 0.8319 - val_loss: 1.0112 - val_cat_output_loss: 0.1227 - val_num_output_loss: 0.8884 - val_cat_output_acc: 0.9396 - val_num_output_acc: 0.7837
Epoch 14/20
20029/2002

Epoch 15/20
20029/20029 [==============================] - 1s 54us/step - loss: 0.4531 - cat_output_loss: 0.1191 - num_output_loss: 0.3340 - cat_output_acc: 0.9405 - num_output_acc: 0.8401 - val_loss: 1.2288 - val_cat_output_loss: 0.1202 - val_num_output_loss: 1.1086 - val_cat_output_acc: 0.9410 - val_num_output_acc: 0.7941
Epoch 16/20
20029/20029 [==============================] - 1s 54us/step - loss: 0.5414 - cat_output_loss: 0.1191 - num_output_loss: 0.4223 - cat_output_acc: 0.9404 - num_output_acc: 0.8416 - val_loss: 1.1095 - val_cat_output_loss: 0.1203 - val_num_output_loss: 0.9892 - val_cat_output_acc: 0.9397 - val_num_output_acc: 0.7883
Epoch 17/20
20029/20029 [==============================] - 1s 54us/step - loss: 0.5794 - cat_output_loss: 0.1204 - num_output_loss: 0.4590 - cat_output_acc: 0.9392 - num_output_acc: 0.8372 - val_loss: 1.0458 - val_cat_output_loss: 0.1210 - val_num_output_loss: 0.9248 - val_cat_output_acc: 0.9400 - val_num_output_acc: 0.8015
Epoch 18/20
20029/2002

In [ ]:
new_sheet

In [ ]:
sheet_name_csv = './test_kor_' +  str(epochs) + '.csv'
with open(sheet_name_csv, 'wb') as f:
        np.savetxt(f,  new_sheet,  delimiter=",", fmt='%s', encoding='cp949')

In [ ]:
save_result(sheet_name_csv)